In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from src.candidate_generation import CandidateGenerationClassification

### Main code

In [ ]:
# Instantiate a CandidateGeneration object, which requires to pass the dataframes of (1) the labels given to the users and (2) some mapping
# between the cells of a grid and the users.
path_users_classification = './data_simulator/huge_dataset/dataset_simulator_trajectories.compressed.parquet.classified.parquet'
path_users_cells_mapping = './mapping_users_cells_grid_50m.pkl'

cand_gen = CandidateGenerationClassification(path_users_classification, path_users_cells_mapping)
print(f"Global positive rate: {cand_gen.get_global_positive_rate()}")

### Various operations

In [ ]:
cnt_threshold = 5
eps_threshold = 0.1
cand_gen.candidate_generation(cnt_threshold, eps_threshold)

In [ ]:
# Codice generico per intersezione liste ID utenti di combinazione celle con lista utenti cella singola.
# TODO: da mettere a posto anche per il caso in cui dim_itemset=1 (caso base) e poi incapsulare in una classe.
dim_itemset = res_intersections.index.nlevels

base_lvls = list(range(dim_itemset - 1))
intersections = {}
for keys, sub in res_intersections.loc[:, 'list_users'].groupby(level=base_lvls, sort=False) :
    # If the multi-index has size 2, then the first "n-1" levels is just one level, and thus isn't a tuple.
    if dim_itemset - 1 == 1 : keys = (keys,)

    # Drop the first 'n-1' levels in the multi-index, leaving only the last one.
    s = sub.droplevel(base_lvls)
    # display(s)

    # Generate all the possible tuples of length 'n+1' by keeping fixed the first "n-1" keys and 
    # consider all the possible combinations of length 2 that can be generated in the last level of the
    # multi-index.
    for a, b in combinations(s.index, 2):
        # Compute the set intersection, and its cardinality.
        intersection = s[a] & s[b]
        cnt = len(intersection)

        # Add to the dictionary only the cell pairs that have at least 'threshold' users in common.
        # The threshold should be calculated according to the statistical power we want to have in the hypotesis tests.
        if cnt > cnt_threshold : intersections[(*keys, a, b)] = intersection


new_res_intersections = pd.Series(data = intersections, name='list_users').to_frame()
del intersections
display(new_res_intersections)